# 1D Advektion-Dispersion
Analytische Lösung

Basierend auf einem XLS-Worksheet von Prof. Rudolf Liedl \
letzte Änderung 2021 12 08 durch Thomas.Reimann@tu-dresden.de

Aktueller Stand der der Programmierung:

-
- Text ergänzen (TR)

In [1]:
# %matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
from scipy import special
import numpy as np
from ipywidgets import *

Daten und Parameter

In [24]:
#FUNCTIONS FOR COMPUTATION; ADS = ADVECTION, DISPERSION AND SORPTION - EVENTUALLY SET RETARDATION TO 1 FOR NO SORPTION

def IC(PE,r_time):
    IC1 = np.sqrt(0.25*PE/r_time)*(1-r_time)
    if (IC1>0):
        IC2 = 1-(1-special.erfc(abs(IC1)))
    else:
        IC2 = 1+(1-special.erfc(abs(IC1)))
    IC3 = np.sqrt(0.25*PE/r_time)*(1+r_time)
    if (IC3>0):
        IC4 = 1-(1-special.erfc(abs(IC3)))
    else:
        IC4 = 1+(1-special.erfc(abs(IC3)))
    if IC4 == 0:
        IC5 = IC2
    else:
        IC5 = IC2+np.exp(PE)*IC4
    IC  = 1-0.5*IC5
    return IC

def BC(PE,r_time, r_dur):
    # BCx ist positiver puls
    BC1 = np.sqrt(0.25*PE/r_time)*(1-r_time)
    if (BC1>0):
        BC2 = 1-(1-special.erfc(abs(BC1)))
    else:
        BC2 = 1+(1-special.erfc(abs(BC1)))
    BC3 = np.sqrt(0.25*PE/r_time)*(1+r_time)
    BC4 = special.erfc(BC3)
    if BC4 == 0:
        BC5 = BC2
    else:
        BC5 = BC2 + np.exp(PE) * BC4
    
    # BCCx ist negativer puls
    if r_time > r_dur:
        BCC1 = np.sqrt(0.25 * PE / (r_time - r_dur)) * (1 - (r_time - r_dur))
        if BCC1 > 0:
            BCC2 = 1 - (1 - special.erfc(abs(BCC1)))
        else:
            BCC2 = 1 + (1 - special.erfc(abs(BCC1)))
            
        BCC3 = np.sqrt(0.25 * PE / (r_time - r_dur)) * (1 + (r_time - r_dur))
        BCC4 = special.erfc(BCC3)
        if BCC4 == 0:
            BCC5 = BCC2
        else:
            BCC5 = BCC2 + np.exp(PE) * BCC4
            
    if r_time <= r_dur:
        BC = 0.5 * BC5
    else:
        BC = 0.5 * (BC5 - BCC5)
    return BC
    

def transport(l,t1,ci,c0,m,Q,n,a):
    # l : länge der säule, float
    # t1 : versuchsende, float
    # ci : initiale konzentration, float
    # c0 : eingabekonzentration, float
    # m : eingabemasse, float
    # Q : durchfluss, float
    # n : porosität, float
    # a : dispersivität (a > 0), float
    
    # Data for plotting
    t0 = 10       #Startzeit
    dt = 10      #Zeitdiskretisierung
    r  = 2    #Radius der Säule
    
    #Berechnung Zwischenergebnisse
    A =     np.pi*r**2
    q =     Q/A
    v =     q/n
    D =     a*v
    PE =    l/a
    dur =   m/Q/(c0-ci)
    tPV =   l/v
    r_dur = dur/tPV
    r_dt =  dt/tPV

    #Festlegung Zeitbereich
    t = np.arange(t0, t1, dt)

    #Berechnung Konzentration - Klammerterme
    #Set fraction of distance
    r_time = []
    time   = []
    conc   = []

    #compute concentration  
    for t in range(t0, t1, dt):      
        # ADVECTION-DISPERSION
        r_time = t/tPV
        c = ci*IC(PE,r_time)+c0*BC(PE,r_time, r_dur)
        
        conc.append(c)
        time.append(t)
        
# PLOT HERE        
    fig = plt.figure(figsize=(9,6))
    fig, ax = plt.subplots()
    ax.plot(time,conc)
    # plt.ylim(0, ci+c0)
    plt.xlim(0,t1)
    ax.set(xlabel='time', ylabel='conc.',
       title='A-D transport')
    ax.grid()
    plt.legend(('1'), loc=4)
    
# COMMENTED THIS OUT - OPTIONAL SAVE IF INTENDED    
   # fig.savefig("test.png")
    plt.show()
    print(A, q, v, D, PE, dur, tPV, r_dur, r_dt)
#THE FOLLOWING ALLOWS TO DEFINE THE RANGE OF SLIDERS DEPENDING ON OTHER SLIDER SETTINGS (HERE DISP AS FUNCTION OF LENGTH)
    
a_widget = FloatText(value=1.5,min=0.005, max=10,step=0.1,description='DISP 1', readout_format='.3f' )
l_widget  = FloatText(value=15,min=1, max=100,step=1,description='LENGTH', readout_format='.2f' )

def update_a_range(*args):
    a_widget.min = 0.00015 * l_widget.value
    a_widget.max = l_widget.value
    a_widget.step = 0.01*l_widget.value
    
# l_widget.observe(update_D_range, 'value')

interact(transport,
         #DEFINE THE RANGE SUCH THAT NOT TOO MANY COMPUTATIONS ARE NECESSARY WHEN MOVING THE SLIDER!
         l  = l_widget,
         t1 = widgets.IntText(value=1800, min = 60, max = 86400, step = 60, description = 'MAX TIME'),
         ci = widgets.FloatText(value=0,min=0,max=10,step=0.1, description='C initial',readout_format='.2f'),
         c0 = widgets.FloatText(value=0.1,min=0, max=5,step=0.1,description='C0', readout_format='.2f' ),
         m  = widgets.FloatText(value=10,min=0, max=1000,step=1,description='m', readout_format='.2f' ),
         Q  = widgets.FloatText(value=0.2, min=0.00001, max=0.001, step=0.0001, description='Discharge Q',readout_format='.5f'),
         n  = widgets.FloatText(value=0.2,min=0.02, max=0.6, step=0.01, description='PORO 1',readout_format='.2f'),       
         a  = a_widget,
        )

interactive(children=(FloatText(value=15.0, description='LENGTH', step=1.0), IntText(value=1800, description='…

<function __main__.transport(l, t1, ci, c0, m, Q, n, a)>